In [3]:
import numpy as np
import random
import math

In [22]:
def rho(x,p):
    ans = (1-p)*math.log2(1+(1-2*p)*x) + p*math.log2(1-(1-2*p)*x)
    return ans

# Variable Length BZ Algorithm
# takes input array ar, tolerated prob Pe, and crossover prob p
def VLBZ(ar,num,Pe,p):
    # current step
    k = 0
    n = len(ar)
    if(n==1):
        return 0 if num<ar[0] else 1
    q = np.ones(n)/n
    Z = 1/(1-q)
    while np.max(Z)<=math.log2(1/Pe):
        k = k + 1
        sum = 0
        j = -1
        while sum<=0.5:
            print(sum,j)
            j = j + 1
            sum = sum + q[j]
        t1 = 2*(1-sum+q[j]-0.5)
        t2 = 2*(sum-0.5)
        pi1 = (rho(t2,p)-rho(-t2,p))/(rho(t1,p)-rho(-t1,p)+rho(t2,p)-rho(-t2,p))
        sub = 1 if random.random()<pi1 else 0
        j_star = j - sub
        t = 2*(sum-ar[j])-1 if j_star==j-1 else 2*(sum)-1
        Y = 1 if ar[j_star]<num else 0
        N = 1 if random.random()<p else 0
        Y = Y ^ N
        f1 = np.ones(n)
        cond = np.ones(n,dtype=bool)
        for i in range(n):
            q[i]=(f1[i] if ((j_star<=i) == (Y)) else f1[i]*p/(1-p))
        Z = q/(1-q)
    
    l = np.argmax(q)
    return l

In [28]:
print('ans = ',str(VLBZ([1,3,8,12],7,0.2,0.3)))

0 -1
0.25 0
0.5 1
ans =  1


/tmp/ipykernel_3918/2367448459.py:36: RuntimeWarning: divide by zero encountered in true_divide
  Z = q/(1-q)
